# Nuestras Caras

Este código implementa una técnica de reconocimiento facial conocida como "Eigenfaces". 

La idea básica detrás de Eigenfaces es representar las caras de un conjunto de datos de entrenamiento en un espacio de características de dimensionalidad reducida utilizando el análisis de componentes principales (PCA). 

Luego, las nuevas caras se proyectan en este espacio de características y se comparan con las caras de entrenamiento para reconocerlas.

DEPENDENCIAS NECESARIAS:

Previo a ejecutar la notebook deben ser instaladas las siguientes dependencias:
* pip install opencv-python
* pip install pandas
* pip install matplotlib
* pip install scikit-learn

In [8]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from photo_30x30 import cortar_imagenes
from pixels import intensidad_pixels
import perceptrones as perc

In [2]:
import re
def limpiar_string(string, limpiar=True):
    if limpiar:
        # Eliminar números seguidos de '.jpg'
        string = re.sub(r'\d+\.jpg', '', string)
        # Eliminar números seguidos de 'jpeg'
        string = re.sub(r'\d+\.jpeg', '', string)
        return string
    else:
        return string
    
def es_imagen_1(string):
    # Utilizar expresión regular para buscar '1.jp' sin otro '1' precediéndolo
    return bool(re.search(r'(?<!1)1\.jp', string))

In [9]:
#********************************************************************************************************************
#                    CORTE DE LAS FOTOS Y CAMBIO DE ESCALA DE GRISES MEDIANTE PHOTO_30X30.PY
#********************************************************************************************************************
# Define the folder path
# Get the current working directory
#current_directory = os.getcwd()
folder_name_raw = "fotos_crudas"
folder_name_output = "fotos_output"

# current_directory
# os.chdir(os.path.join(current_directory, "NuestrasCaras"))

folder_path_output = os.path.join(os.getcwd(), folder_name_output)

# Cortar las fotos y cambiar a escala de grises
cortar_imagenes(folder_name_raw, folder_name_output)

#********************************************************************************************************************
#                    OBTENCION DE LOS VALORES DE LOS PIXELES DE LAS FOTOS MEDIANTE PIXELS.PY
#********************************************************************************************************************
# Guardo la info de las fotos en un dataframe
data_fotos = intensidad_pixels(folder_path_output)

# Get the file names
file_names = data_fotos.iloc[:, 0]

# Tomo los nombres de cada persona
people_names = [name.split("-")[0] for name in file_names]

# Get the greyscale values
greyscale_values = data_fotos.iloc[:, 1:].values

KeyboardInterrupt: 